In [1]:
import sys,os
sys.path.append('../')
from smodels.tools import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'
import numpy as np
from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.tools.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
from smodels.tools.coverage import Uncovered,UncoveredGroup
setLogLevel("error")
%load_ext line_profiler
# Set the path to the database
database = Database(os.path.expanduser("../test/database"))

### Load model

In [2]:
slhafile = '../inputFiles/slha/lightEWinos.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


### Decompose

In [3]:
# Set main options for decomposition
sigmacut = 1*fb
mingap = 5.*GeV

# Decompose model
topDict = decomposer.decompose(model, sigmacut,
                               massCompress=True, invisibleCompress=True,
                               minmassgap=mingap)

In [4]:
print(len(topDict.getElements()))
totWeight = None
for el in topDict.getElements():
    if totWeight is None:
        totWeight = el.weightList.copy()
    else:
        totWeight += el.weightList
print(totWeight.getMaxXsec())

917
1.40E+01 [pb]


### Load the experimental results to be used.

In [5]:
listOfExpRes = database.getExpResults()

### Compute the theory predictions

In [6]:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
allPredictions = []
for expResult in listOfExpRes:
    predictions = theoryPredictionsFor(expResult, topDict, combinedResults=False, marginalize=False)
    if not predictions:
        continue  # Skip if there are no constraints from this result
    print('\n %s ' % expResult.globalInfo.id)
    for theoryPrediction in predictions:
        # Compute the r-value
        r = theoryPrediction.getRValue()
        print("r = %1.3E" % r)


 Theory Predictions and Constraints:

 ATLAS-CONF-2013-037 
r = 1.142E-01

 ATLAS-SUSY-2013-02 
r = 6.560E-05

 ATLAS-SUSY-2013-12 
r = 6.460E-01

 CMS-SUS-13-012 
r = 5.050E-03


In [7]:
# import time
# t0 = time.time()
uncovered = Uncovered(topDict, sqrts=13.*TeV)
# print(time.time()-t0)

In [8]:
# First sort coverage groups by label
groups = sorted(uncovered.groups[:], key=lambda g: g.label)
# Print uncovered cross-sections:
for group in groups:
    print("\nTotal cross-section for %s (fb): %10.3E\n" % (group.description, group.getTotalXSec()))


Total cross-section for missing topologies (fb):  1.046E+04


Total cross-section for missing topologies with displaced decays (fb):  0.000E+00


Total cross-section for missing topologies with prompt decays (fb):  1.046E+04


Total cross-section for topologies outside the grid (fb):  8.055E+01



In [9]:
missingTopos = uncovered.getGroup('missing (prompt)')
# Print some of the missing topologies:
finalStates = {}
for iel,genEl in enumerate(missingTopos.finalStateElements[:]):
    elStr = str(genEl)
    if elStr not in finalStates:
        finalStates[elStr] = genEl.missingX
    else:
        finalStates[elStr] += genEl.missingX

            # Print some of the missing topologies:
print(len(finalStates))
for fs in sorted(finalStates.keys()):
    print('Element:', fs)
    print('\tcross-section (fb):', np.round(finalStates[fs],4))        

148
Element: (PV > MET,MET)
	cross-section (fb): 3.8833
Element: (PV > MET,MET,Z)
	cross-section (fb): 1.8942
Element: (PV > MET,MET,b,b)
	cross-section (fb): 1.1665
Element: (PV > MET,MET,b,b,b,b,Z)
	cross-section (fb): 2.5528
Element: (PV > MET,MET,b,b,b,b,b,b)
	cross-section (fb): 67.8715
Element: (PV > MET,MET,b,b,b,b,b,b,b,b)
	cross-section (fb): 15.1209
Element: (PV > MET,MET,higgs,b,b,b,b)
	cross-section (fb): 2.3918
Element: (PV > MET,MET,jet,b,b)
	cross-section (fb): 2.1883
Element: (PV > MET,MET,jet,b,b,Z)
	cross-section (fb): 45.6887
Element: (PV > MET,MET,jet,b,b,b,b)
	cross-section (fb): 3.993
Element: (PV > MET,MET,jet,b,b,b,b,Z)
	cross-section (fb): 27.0839
Element: (PV > MET,MET,jet,b,b,b,b,b,b)
	cross-section (fb): 2.2603
Element: (PV > MET,MET,jet,b,b,b,b,b,b,Z)
	cross-section (fb): 3.4521
Element: (PV > MET,MET,jet,higgs,b,b)
	cross-section (fb): 16.1158
Element: (PV > MET,MET,jet,higgs,b,b,b,b)
	cross-section (fb): 11.4513
Element: (PV > MET,MET,jet,higgs,l,nu,t,b)


In [10]:
missingDisplaced = uncovered.getGroup('missing (displaced)')
# Print elements with displaced decays:
if missingDisplaced.finalStateElements:
    for genEl in missingDisplaced.finalStateElements[:3]:
        print('Element:', genEl)
        print('\tcross-section (fb):', np.round(genEl.missingX,4))
else:
    print("\nNo displaced decays")



No displaced decays


In [11]:
missingGrid = uncovered.getGroup('outsideGrid (all)')
finalStates = {}
for iel,genEl in enumerate(missingGrid.finalStateElements[:]):
    elStr = str(genEl)
    if elStr not in finalStates:
        finalStates[elStr] = genEl.missingX
    else:
        finalStates[elStr] += genEl.missingX

print(len(finalStates))
for fs in sorted(finalStates.keys()):
    print('Element:', fs)
    print('\tcross-section (fb):', np.round(finalStates[fs],4))        

2
Element: (PV > MET,MET,b,b,b,b)
	cross-section (fb): 72.7112
Element: (PV > MET,MET,t,t,b,b)
	cross-section (fb): 7.8367
